import multiprocessing<br>
import tqdm<br>
from pickle import FALSE, TRUE<br>
import matplotlib.pyplot as plt

In [ ]:
from cmath import exp
import random
import numpy as np
# import scipy.stats as st
# import numba
# import biocircuits

Plotting modules

In [ ]:
import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

In [ ]:
from bokeh.models import Band, ColumnDataSource
import pandas as pd

Line profiler (can install with conda install line_profiler)<br>
%load_ext line_profiler

In [ ]:
def boundtime_dep_propensity(lifetimes, kbind, kunbind):
    # Three scenarios - 0) new pMHC binds, 1) bound pMHC unbinds, 2) bound pMHC forms LAT
    nBoundPMhc = lifetimes.size
    propensities = np.array([]) 
    propensities = np.append(propensities, kbind)  # Scenario 0)
    propensities = np.append(propensities, kunbind * nBoundPMhc) # Scenario 1)
    if lifetimes.size > 0: # Scenario 2)
        for t in np.nditer(lifetimes):
            kN = 0.1
            # N = 1
            kt = np.square(kN) * t * np.exp(-2*kN*t) # This is a test in the time being. %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            # kt = 0.1 * t
            propensities = np.append(propensities, kt)
    return propensities

Specify parameters for calculation

In [ ]:
time_points = np.linspace(0, 50, 101)
kN = 0.1
y = [np.square(kN) * t * np.exp(-2*kN*t) for t in time_points] 
plotkt = bokeh.plotting.figure(plot_width=300,plot_height=200,
                               x_axis_label='dwell time',y_axis_label='nucleation rate')
plotkt.line(time_points, y,        line_width=3, color='gray', line_join='bevel')
bokeh.io.show(plotkt)

In [ ]:
kbind = 1
kunbind = 1 
args = (kbind, kunbind)

In [ ]:
nCell = 100
# np.random.seed(42) # Seed random number generator for reproducibility

Initialize output array

In [ ]:
samples = np.empty((nCell, len(time_points), 2), dtype=int)

In [ ]:
pMhcLifeLogs = np.array([])
# Run the calculations
for i in range(nCell): #tqdm.notebook.tqdm(range(size)):
    samples[i,:,:], temp = gillespie_ssa(boundtime_dep_propensity, time_points, args=args)
    pMhcLifeLogs = np.append(pMhcLifeLogs, temp)
    pMhcLifeLogs = np.reshape(pMhcLifeLogs, (-1,len(time_points)))

In [ ]:
plot2 = bokeh.plotting.figure(plot_width=300,plot_height=200,
                               x_axis_label='dwell time',y_axis_label='number of pMHC', 
                               y_axis_type="log")
finalValidTimeIdx = np.argwhere(np.sum(~np.isnan(pMhcLifeLogs), axis=0)==0).min() 
y = pMhcLifeLogs[:,:finalValidTimeIdx]
x = time_points[:finalValidTimeIdx]
plot2.line(x, sum(~np.isnan(y)),        line_width=3, color='pink', line_join='bevel')
bokeh.io.show(plot2)

Set up plots

In [ ]:
countmsg = 'N = %d' % len(pMhcLifeLogs)
plots = [bokeh.plotting.figure(plot_width=300,plot_height=200,x_axis_label='time',
                                y_axis_label='number of bound pMHC', title=countmsg),
         bokeh.plotting.figure(plot_width=300,plot_height=200,x_axis_label='time',
                                y_axis_label='number of LAT condensates', title=countmsg)]
# Plot trajectories and mean
for i in [0, 1]: # 0 is nBoundPMhc and 1 is nLat
    for x in samples[:,:,i]: # x is each trial sequence
        plots[i].line(time_points, x, line_width=0.6,     alpha=0.6, line_join='bevel')
    plots[i].line(time_points, samples[:,:,i].mean(axis=0),    
                    line_width=3, color='orange', line_join='bevel')
# Link axes
plots[0].x_range = plots[1].x_range
bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=2))

In [ ]:
plot3 = bokeh.plotting.figure(plot_width=300,plot_height=200,
                                x_axis_label='dwell time',y_axis_label='prob(LAT)')
finalValidTimeIdx = np.argwhere(np.sum(~np.isnan(pMhcLifeLogs), axis=0)<1).min() 
yy = pMhcLifeLogs[:,:finalValidTimeIdx]
x = time_points[:finalValidTimeIdx]

In [ ]:
for y in yy[:,:]: # pMHC, record | x: per-pMHC
    plot3.line(x, y, line_width=0.6,             alpha=0.6, line_join='bevel')

In [ ]:
plot3.line(x, np.nanmean(yy[:,:], axis=0),       line_width=3, color='green', line_join='bevel')

create the coordinates for the errorbars

In [ ]:
sem = np.nanstd(yy[:,:], axis=0) / np.sqrt(sum(~np.isnan(yy)))
err_xs = []
err_ys = []

In [ ]:
for x, y, yerr in zip(x, np.nanmean(yy[:,:], axis=0), sem):
    err_xs.append((x, x))
    err_ys.append((y - yerr, y + yerr))

plot them

In [ ]:
plot3.multi_line(err_xs, err_ys, color='green')
plot3.x_range.start = time_points[0]
plot3.x_range.end = time_points[-1]
bokeh.io.show(plot3)

In [ ]:
plot1 = plot3
finalValidTimeIdx = np.argwhere(np.sum(~np.isnan(pMhcLifeLogs), axis=0)<10).min() 
plot1.x_range.start = time_points[0]
plot1.x_range.end = time_points[finalValidTimeIdx-2]
bokeh.io.show(plot1)

In [ ]:
def common_member(a, b):   
    a_set = set(a)
    b_set = set(b)
     
    # check length
    if len(a_set.intersection(b_set)) > 0:
        return a_set.intersection(b_set)
    else:
        return []

In [ ]:
def sample_discrete(probs):
    # Generate random number
    q = np.random.rand()
    # Find index
    i = 0
    p_sum = 0.0
    while p_sum < q:
        p_sum += probs[i]
        i += 1
    return i - 1

In [ ]:
def gillespie_draw(propensity_func, lifetimes, args=()):
    # Compute propensities
    calc_propensities = propensity_func(lifetimes, *args)
    # Sum of propensities
    props_sum = calc_propensities.sum()
    # Compute next time
    time = np.random.exponential(1.0 / props_sum)
    # Compute discrete probabilities of each reaction
    rxn_probs = calc_propensities / props_sum
    # Draw reaction from this distribution
    rxn = sample_discrete(rxn_probs)
    return rxn, time